## Finding the Ground State Energy of Molecular Hydrogen (H2) via the Variational Quantum Eigensolver (VQE).

VQE is the most commonly used algorithm to obtain the ground state energy of molecules due to lower requirements in the coherence times of the Quantum Processing Unit (QPU). 

As explained in the theory chapter it combines the quantum and classical processes. Specifically, it will use the QPU to measure the expectation value of the Molecular Hamiltonian ($\langle \Psi | H | \Psi \rangle$) and a classical optimization method to vary the parameters of the Wavefunction (commonly used optimizers are SLSQP or SPSA).

Let us see how do we obtain the Ground State Energy of H2 using Qiskit.

In [1]:
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver

driver = PySCFDriver(
    atom="H 0 0 0; H 0 0 0.735",
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)

es_problem = driver.run()

Numpy 1.16 has memory leak bug  https://github.com/numpy/numpy/issues/13808
It is recommended to downgrade to numpy 1.15 or older


The code above has used a classical Quantum Chemical Software (PySCF) to obtain the Hartree-Fock wavefunction.
We can easily visualize the matrix elements of the Molecular Hamiltionian by accessing the properties of the object `es_problem`

In [6]:
ham_matrix = es_problem.hamiltonian.second_q_op().to_matrix()
print(ham_matrix)

  (1, 1)	(-0.4718960072811405+0j)
  (2, 2)	(-1.2563390730032507+0j)
  (3, 3)	(-1.2445845498133257+0j)
  (4, 4)	(-0.4718960072811405+0j)
  (5, 5)	(-0.2452182918302624+0j)
  (10, 5)	(0.1809311997842315+0j)
  (6, 6)	(-1.0636533500290941+0j)
  (9, 6)	(0.18093119978423156+0j)
  (7, 7)	(-0.35332510410715035+0j)
  (8, 8)	(-1.2563390730032507+0j)
  (6, 9)	(0.18093119978423156+0j)
  (9, 9)	(-1.0636533500290941+0j)
  (5, 10)	(0.18093119978423142+0j)
  (10, 10)	(-1.8369679912029842+0j)
  (11, 11)	(-1.1606317377577622+0j)
  (12, 12)	(-1.2445845498133257+0j)
  (13, 13)	(-0.3533251041071502+0j)
  (14, 14)	(-1.1606317377577617+0j)
  (15, 15)	(0.2142782384194788+0j)


In order to transform this problem into something that the QPU can operate with, it is needed to change basis from the Fock Space to Pauli basis. The specific transformation is called mapper or encoder. Let's use one of the more extended mappers as is the `JordanWignerMapper`

In [14]:
from qiskit_nature.second_q.mappers import JordanWignerMapper, QubitConverter

mapper = QubitConverter(JordanWignerMapper()) 

And now, let us define the solver for finding the lowest eigenvalue of the Hamiltonian with respect to the parametrized wavefunction. At this point we also need to define the quantum circuit that we want to use. In this case, since the molecule and basis set are small, we can opt for a chemically ispired ansatz as Unitary Coupled-Cluster (UCC).

In [16]:
from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.algorithms.optimizers import SLSQP
from qiskit.primitives import Estimator
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD

initial_state = HartreeFock(es_problem.num_spatial_orbitals,es_problem.num_particles,mapper)

ansatz = UCCSD(
    es_problem.num_spatial_orbitals,
    es_problem.num_particles,
    mapper,
    initial_state=initial_state
)

vqe_solver = VQE(Estimator(), ansatz, SLSQP())
vqe_solver.initial_point = [0.0] * ansatz.num_parameters

Once we have defined all the necessary ingredients for obtaining the ground state energy we can use the overall algorithm `GroundStateEigensolver` that wraps all the above defined variables and algos into a single and compact algorithm.

In [17]:
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

calc = GroundStateEigensolver(mapper, vqe_solver)
res = calc.solve(es_problem)
print(res)

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -1.857275030144
  - computed part:      -1.857275030144
~ Nuclear repulsion energy (Hartree): 0.719968994449
> Total ground state energy (Hartree): -1.137306035695
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  1.3889487]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.0  1.38894846]
    - computed part:      [0.0  0.0  1.38894846]
  > Dipole moment (a.u.): [0.0  0.0  0.00000024]  Total: 0.00000024
                 (debye): [0.0  0.0  0.00000061]  Total: 0.00000061
 
